## **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import * 

df = spark.read.format("parquet")\
                .load("abfss://bronze@storagetosty.dfs.core.windows.net/customers")

display(df)

## **Column Transformation**

In [0]:
df = df.drop("_rescued_data")
df = df.withColumn("domain", split(col('email'), '@').getItem(1))
display(df.select("domain"))
#display(df)

In [0]:
df_agg = df.groupBy("domain").agg(count("customer_id").alias("total")).sort("total", ascending=False)

display(df_agg)

In [0]:
df.filter(df.domain == 'gmail.com').display()
df.filter(df.domain == 'yahoo.com').display()
df.filter(df.domain == 'hotmail.com').display()

In [0]:
df = df.withColumn("full_name", concat(col('first_name'), lit(' '), col('last_name')))
display(df)

## **Data Writting**

In [0]:
df.write.format("delta")\
                .mode("append")\
                .save("abfss://silver@storagetosty.dfs.core.windows.net/customers")

In [0]:
spark.read.format("delta").load("abfss://silver@storagetosty.dfs.core.windows.net/customers").display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS `databricks-catalog`.silver.silver_customers
USING DELTA
LOCATION 'abfss://silver@storagetosty.dfs.core.windows.net/customers'

In [0]:
%sql
SELECT * FROM `databricks-catalog`.silver.silver_customers